In [1]:
import pandas as pd
df = pd.read_csv("Friday-Merged-Binary.csv")
df.head()


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Attempted Category
0,3268,112740690,32,16,6448,1152,403,0,201.5,204.724205,...,3.594286e+02,1.199802e+01,380,343,16100000.0,4.988048e+05,16400000,15400000,BENIGN,-1
1,389,112740560,32,16,6448,5056,403,0,201.5,204.724205,...,3.202857e+02,1.574499e+01,330,285,16100000.0,4.987937e+05,16400000,15400000,BENIGN,-1
2,0,113757377,545,0,0,0,0,0,0.0,0.000000,...,9.361829e+06,7.324646e+06,18900000,19,12200000.0,6.935824e+06,20800000,5504997,BENIGN,-1
3,5355,100126,22,0,616,0,28,28,28.0,0.000000,...,0.000000e+00,0.000000e+00,0,0,0.0,0.000000e+00,0,0,BENIGN,-1
4,0,54760,4,0,0,0,0,0,0.0,0.000000,...,0.000000e+00,0.000000e+00,0,0,0.0,0.000000e+00,0,0,BENIGN,-1


In [2]:

import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import joblib

df = df.rename(columns=lambda x: x.strip())
df = df.dropna()
df = df.drop_duplicates()

X = df.drop(columns=["Label"])
y = df["Label"]


le = LabelEncoder()
y_encoded = le.fit_transform(y)


X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.4, random_state=42)

model = XGBClassifier(
    max_depth=6,
    random_state=0,
    eval_metric="logloss"
)
model.fit(X_train, y_train)


y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=le.classes_))

joblib.dump(model, "cicids_rf_model.pkl")
joblib.dump(le, "cicids_label_encoder.pkl")
joblib.dump(list(X.columns), "cicids_features.pkl")


              precision    recall  f1-score   support

      BENIGN       1.00      1.00      1.00    157986
         Bot       1.00      1.00      1.00       803
        DDoS       1.00      1.00      1.00     51258
    PortScan       1.00      1.00      1.00     36127

    accuracy                           1.00    246174
   macro avg       1.00      1.00      1.00    246174
weighted avg       1.00      1.00      1.00    246174



['cicids_features.pkl']

In [ ]:
# app.py
import streamlit as st
import numpy as np
import pandas as pd
import joblib


model = joblib.load("cicids_rf_model.pkl")
label_encoder = joblib.load("cicids_label_encoder.pkl")
feature_names = joblib.load("cicids_features.pkl")

st.title("🚨 CICIDS Intrusion Detection")
st.write("Enter feature values to detect whether traffic is normal or an attack.")


user_input = []
for feature in feature_names:
    value = st.number_input(f"{feature}", min_value=0.0, max_value=1e6, value=0.0)
    user_input.append(value)

input_array = np.array(user_input).reshape(1, -1)

if st.button("Predict"):
    prediction = model.predict(input_array)[0]
    pred_label = label_encoder.inverse_transform([prediction])[0]
    st.success(f"🛡️ Prediction: {pred_label}")

 
 
    proba = model.predict_proba(input_array)[0]
    proba_df = pd.DataFrame({label_encoder.classes_[i]: [proba[i]] for i in range(len(proba))})
    st.bar_chart(proba_df.T)
 

2025-07-20 13:36:47.975 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-20 13:36:48.042 
  command:

    streamlit run C:\Users\mudit\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-07-20 13:36:48.043 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-20 13:36:48.043 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-20 13:36:48.044 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-20 13:36:48.044 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-20 13:36:48.045 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-20 13:36:48.046 Thre